In [13]:
import cv2
import numpy as np
from skimage.exposure import rescale_intensity

In [14]:
# reference: https://www.youtube.com/watch?v=wmYWfA-4HE4
def convole(image: np.ndarray, kernel: np.ndarray):
    iH, iW = image.shape[:2]
    kH, kW = kernel.shape[:2]
    print(image.shape, kernel.shape)
    
    '''
        "pad" the borders of the input image
        so the spatial size (i.e. width and height) are not reduce
    '''
    pad = (kW - 1) // 2
    print(pad)
    # fill empty space with padding
    image = cv2.copyMakeBorder(src=image,
                               top=pad,
                               bottom=pad,
                               left=pad,
                               right=pad,
                               borderType=cv2.BORDER_REPLICATE)
    output = np.zeros((iH, iW), dtype='float32')
    
    '''
        loop over the input image, "sliding" the kernel
        across each (x,y) coordinate from left to right and top to bottom
    '''
    for y in np.arange(pad, iH + pad):
        for x in np.arange(pad, iW + pad):
            # extract the ROI of the image by extracting the 
            # "center" region of the current (x,y) coordinate dimensions
            roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]
            
            # perform the convolution by taking 
            # the matrix multiplicate between ROI and kernel
            # then sum the matrix
            k = (roi * kernel).sum()
            
            # store value in the (x,y) coordinate of the output image
            output[y - pad, x - pad] = k
    
    # rescale the value from binary (0,1) to (0,255) range
    output = rescale_intensity(output, in_range=(0, 255))
    output = (output * 255).astype('uint8')
    
    return output

In [15]:
small_blur = np.zeros((1,1), dtype="float") * (1.0 / (1 * 1))
large_blur = np.zeros((7,7), dtype='float')

# sharpening filter
sharpen = np.array((
    [0, -1, 0],
    [-1, 5, -1],
    [0, -1, 0]
), dtype='int')

# edge like filter
laplacian = np.array((
    [0, 1, 0],
    [1, -4, 1],
    [0, 1, 0]
), dtype='int')

sobel_x = np.array((
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
), dtype='int')

sobel_y = np.array((
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]
), dtype='int')

kernel_bank = (
    ('small_blur', small_blur),
    ('large_blur', large_blur),
    ('sharpen', sharpen),
    ('laplacian', laplacian),
    ('sobel_x', sobel_x),
    ('sobel_y', sobel_y)
)

In [16]:
image = cv2.imread('../resources/car.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

for name, kernel in kernel_bank:
    convole_output = convole(gray, kernel)
    opencv_output = cv2.filter2D(gray, -1, kernel)
    
    cv2.imshow('origin', gray)
    cv2.imshow('{}-convole'.format(name), convole_output)
    cv2.imshow('{}-opencv'.format(name), opencv_output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()    

(742, 1200) (7, 7)
3
